### IMPORT THƯ VIỆN

In [ ]:
import requests
import re
import csv
import time
import html 
from bs4 import BeautifulSoup
from typing import Optional, List, Tuple, Dict
from urllib.parse import urljoin

### Thu thập dữ liệu từ thanglong.chinhphu.vn

In [ ]:
import requests
import re
import csv
import time
import html
from bs4 import BeautifulSoup
from typing import Optional, Tuple, List, Dict
from urllib.parse import urljoin

# --- CẤU HÌNH ---
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36',
})

DOMAIN = 'https://thanglong.chinhphu.vn'
# Regex bắt link bài viết: thường kết thúc bằng -số_id.htm (VD: ...-10223.htm)
ARTICLE_URL_RE = re.compile(r'-(\d+)\.htm[l]?(?:$|\?)')

# --- CÁC HÀM XỬ LÝ TEXT ---
def clean_text(s: str) -> str:
    if not s: return ''
    s = html.unescape(s)
    s = html.unescape(s)
    return re.sub(r'\s+', ' ', s).strip()

def remove_prefix(text: str) -> str:
    pattern = r'^\s*\(Chinhphu\.vn\)\s*-\s*'
    return re.sub(pattern, '', text, flags=re.IGNORECASE).strip()

def normalize_href(href: str, base_url: str) -> Optional[str]:
    if not href or href.startswith(('javascript:', '#', 'mailto:')): return None
    if href.startswith('//'): return 'https:' + href
    if href.startswith('/'): return urljoin(DOMAIN, href)
    if href.startswith('http'): return href
    return urljoin(base_url, href)

# --- [SỬA ĐỔI QUAN TRỌNG] HÀM TẠO URL PHÂN TRANG ---
def make_page_url(base: str, page: int) -> str:
    """
    Chuyển đổi URL danh mục sang URL trang 2, 3...
    VD: .../thoi-su.htm -> .../thoi-su/trang-2.htm
    """
    if page <= 1: return base
    
    # 1. Bỏ đuôi .htm hoặc .html
    base_no_ext = re.sub(r'\.htm[l]?$', '', base, flags=re.IGNORECASE)
    
    # 2. Thêm đuôi /trang-{page}.htm
    return f"{base_no_ext}/trang-{page}.htm"

# --- HÀM CRAWL CHI TIẾT ---
def parse_article(url: str) -> Tuple[str, str, str, str, str, str]:
    try:
        resp = session.get(url, timeout=15)
        if resp.status_code != 200: return '', '', '', '', '', ''
        
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'html.parser')

        # 1. URL
        canonical = soup.find('link', attrs={'rel': 'canonical'})
        article_url = canonical.get('href') if canonical else url

        # 2. TITLE
        meta_title = soup.find('meta', property='og:title')
        title = clean_text(meta_title.get('content')) if meta_title else ''
        if not title:
            t_tag = soup.find('title')
            title = clean_text(t_tag.get_text()) if t_tag else ''

        # 3. PUBLISH TIME
        time_tag = soup.find(attrs={"data-role": "publishdate"})
        publish_time = clean_text(time_tag.get_text()) if time_tag else ''
        if not publish_time:
             meta_time = soup.find('meta', property='article:published_time')
             if meta_time: publish_time = clean_text(meta_time['content'])

        # 4. KEYWORDS
        meta_kw = soup.find('meta', attrs={'name': 'news_keywords'})
        if not meta_kw: meta_kw = soup.find('meta', attrs={'name': 'keywords'})
        keywords = clean_text(meta_kw.get('content')) if meta_kw else ''

        # 5. SUMMARY
        meta_desc = soup.find('meta', property='og:description')
        summary = clean_text(meta_desc.get('content')) if meta_desc else ''
        summary = remove_prefix(summary)

        # 6. CONTENT
        content = ''
        body = soup.select_one('div.detail-content, div.afcbc-body')
        
        if body:
            for garbage in body.select('table, figure, script, style, .box-responsive, .relate-news, .box-ads'):
                garbage.decompose()

            paras = body.select('p')
            
            # Xóa chữ ký tác giả ở cuối nếu ngắn
            if paras:
                last_txt = clean_text(paras[-1].get_text())
                if len(last_txt) < 50 and len(last_txt) > 0:
                     paras.pop()

            cleaned_paras = []
            for i, p in enumerate(paras):
                txt = clean_text(p.get_text())
                if not txt: continue
                # Bỏ tiền tố (Chinhphu.vn) ở đoạn đầu
                if i == 0: txt = remove_prefix(txt)
                cleaned_paras.append(txt)
            
            content = ' '.join(cleaned_paras)

        return title, summary, content, keywords, publish_time, article_url

    except Exception as e:
        print(f"Lỗi đọc bài ({url}): {e}")
        return '', '', '', '', '', ''

def get_links(url):
    try:
        resp = session.get(url, timeout=15)
        soup = BeautifulSoup(resp.text, 'html.parser')
        links = set()
        
        # Tìm trong các container chứa tin bài
        # Thêm các class thường gặp: article-list, timeline, zone-content
        main = soup.select_one('div.main, div#admwrapper, body') 
        
        if not main: return []
        
        for a in main.select('a[href]'):
            full = normalize_href(a.get('href'), url)
            
            # ĐIỀU KIỆN LỌC LINK:
            if full and DOMAIN in full:
                # 1. Phải khớp regex (có số ID ở cuối)
                if ARTICLE_URL_RE.search(full):
                    # 2. Không phải chính link đang quét
                    if full != url:
                        # 3. Không chứa 'trang-X.htm' (để tránh nhầm link phân trang là bài viết)
                        if '/trang-' not in full:
                            links.add(full)
                            
        return list(links)
    except: return []

# --- HÀM MAIN ---
def main(topics_dict):
    # [CẤU HÌNH] Tăng số trang muốn quét để lấy nhiều tin hơn
    MAX_PAGES_PER_LINK = 40  
    OUTPUT_FILE = 'thanglong_full_data.csv'

    seen_global = set() # Tránh trùng lặp giữa các trang

    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=['topic', 'title', 'summary', 'content', 'keywords', 'publish_time', 'url'])
        writer.writeheader()
        
        for topic_name, url_data in topics_dict.items():
            print(f"\n========== CHỦ ĐỀ: {topic_name} ==========")
            
            list_urls = [url_data] if isinstance(url_data, str) else url_data

            for sub_url in list_urls:
                print(f"   >>> Đang quét nhánh: {sub_url}")
                
                # Vòng lặp quét các trang 1, 2, 3... (tương ứng ấn 'Xem thêm')
                for p in range(1, MAX_PAGES_PER_LINK + 1):
                    p_url = make_page_url(sub_url, p)
                    print(f"      -> Trang {p}: {p_url}")
                    
                    try:
                        links = get_links(p_url)
                    except:
                        links = []
                    
                    if not links:
                        print("         (Hết bài hoặc không tìm thấy)")
                        break # Dừng vòng lặp trang nếu trang hiện tại ko có bài

                    count_ok = 0
                    for link in links:
                        if link in seen_global: continue
                        seen_global.add(link)
                        
                        # Crawl bài viết
                        title, summary, content, kw, p_time, final_url = parse_article(link)
                        
                        if title and content:
                            writer.writerow({
                                'topic': topic_name,
                                'title': title,
                                'summary': summary,
                                'content': content,
                                'keywords': kw,
                                'publish_time': p_time,
                                'url': final_url
                            })
                            count_ok += 1
                            # In tiến độ 
                            print(f"         ✅ {title[:40]}...")
                        
                        time.sleep(0.1) 
                    
                    print(f"      -> Đã lưu {count_ok} bài mới.")
                    time.sleep(1.0) # Nghỉ giữa các trang phân trang

if __name__ == "__main__":
    # Danh sách chủ đề (đã bao gồm cả dạng chuỗi đơn và dạng danh sách)
    topics = {
        'Thời sự': 'https://thanglong.chinhphu.vn/thoi-su.htm',
        'Kinh tế': [
            'https://thanglong.chinhphu.vn/kinh-te.htm',
            'https://thanglong.chinhphu.vn/kinh-te/ngan-hang.htm',
            'https://thanglong.chinhphu.vn/kinh-te/chung-khoan.htm',
            'https://thanglong.chinhphu.vn/kinh-te/thi-truong.htm',
            'https://thanglong.chinhphu.vn/kinh-te/doanh-nghiep.htm',
            'https://thanglong.chinhphu.vn/kinh-te/khoi-nghiep.htm'
        ],
        'Xã hội': [
            'https://thanglong.chinhphu.vn/xa-hoi/khoa-hoc.htm',
            'https://thanglong.chinhphu.vn/xa-hoi/giao-duc.htm',
            'https://thanglong.chinhphu.vn/xa-hoi/phap-luat.htm',
            'https://thanglong.chinhphu.vn/xa-hoi/y-te.htm',
            'https://thanglong.chinhphu.vn/xa-hoi/doi-song.htm'
        ],
        'Văn hóa': 'https://thanglong.chinhphu.vn/van-hoa-giai-tri/ha-noi-van-hien.htm',
        'Chính phủ': 'https://thanglong.chinhphu.vn/chinh-phu-voi-ha-noi.htm',
        'Du lịch': [
            'https://thanglong.chinhphu.vn/du-lich/am-thuc.htm',
            'https://thanglong.chinhphu.vn/du-lich/diem-den.htm'
        ],
        'Nhà đầu tư': [
            'https://thanglong.chinhphu.vn/nha-dau-tu/chinh-sach-uu-dai.htm',
            'https://thanglong.chinhphu.vn/nha-dau-tu/cai-cach-thu-tuc-hanh-chinh.htm'
        ]
    }

    main(topics)


========== CHỦ ĐỀ: Thời sự ==========
   >>> Đang quét nhánh: https://thanglong.chinhphu.vn/thoi-su.htm
      -> Trang 1: https://thanglong.chinhphu.vn/thoi-su.htm
         ✅ Hà Nội tổ chức lễ hội Xuân Bính Ngọ 2026...
         ✅ Du lịch, Hà Nội, 2026, khách quốc tế...
         ✅ Đại tá Nguyễn Tiến Đạt giữ chức Thủ trưở...
         ✅ Liên minh HTX TP Hà Nội: Đòn bẩy mở thị ...
         ✅ Hà Nội: Kiện toàn 2 Trưởng ban HĐND, Giá...
         ✅ Người dân Thủ đô được khám bác sĩ bệnh v...
         ✅ Nâng vị thế trung tâm công nghiệp, đổi m...
         ✅ Hà Nội chi quà Tết cho người hưởng lương...
         ✅ Hà Nội yêu cầu đưa ngay các dữ liệu đất ...
         ✅ Phát huy vai trò y tế cơ sở – 'lá chắn đ...
         ✅ CPI bình quân của Hà Nội năm 2025 tăng 3...
         ✅ Xóa bỏ ‘chợ cóc’: Cần có lộ trình và bảo...
         ✅ Hà Nội thu hút 76,9 triệu USD vốn FDI tr...
         ✅ Mặt trận - ‘cầu nối’ vững chắc giữa Đảng...
         ✅ Bảo đảm TTATGT phục vụ sơ duyệt, diễn tậ...
         ✅ ‘Bá

### Thu thập dữ liệu từ giaoducthoidai.vn

In [18]:
import requests
import re
import csv
import time
import html
from bs4 import BeautifulSoup
from typing import Optional, List, Tuple, Dict
from urllib.parse import urljoin

# --- CẤU HÌNH ---
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/124.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://giaoducthoidai.vn/',
})

DOMAIN = 'https://giaoducthoidai.vn'
# Regex: Bắt các link bài viết có dạng ...post12345.html
ARTICLE_URL_RE = re.compile(r'post(\d+)\.html', re.IGNORECASE)

# --- TUỲ CHỈNH ĐỂ CRAWL NHIỀU HƠN ---
MAX_PAGES_PER_LINK = 50      # Tăng lên 50 trang mỗi mục (tương đương ấn xem thêm 50 lần)
STOP_AFTER_EMPTY = 5         # Tăng lên 5: Nếu 5 trang liên tiếp không có bài mới thì mới dừng
SLEEP_BETWEEN_REQUESTS = 0.5 
OUTPUT_FILE = 'giaoducthoidai_full_data.csv'

# --- HÀM TIỆN ÍCH XỬ LÝ TEXT ---
def clean_text(s: str) -> str:
    if not s: return ''
    s = html.unescape(s)
    return re.sub(r'\s+', ' ', s).strip()

def remove_prefix_gdtd(text: str) -> str:
    if not text: return ''
    text = re.sub(r'^GD&TĐ\s*-\s*', '', text, flags=re.IGNORECASE)
    return text.strip()

def normalize_href(href: str, base: str = DOMAIN) -> Optional[str]:
    if not href: return None
    href = href.strip()
    if href.startswith(('javascript:', '#', 'mailto:')): return None
    if href.startswith('//'): return 'https:' + href
    if href.startswith('/'): return urljoin(DOMAIN, href)
    if href.startswith('http'): return href
    return urljoin(base, href)

# --- [QUAN TRỌNG] LOGIC TẠO URL PHÂN TRANG ---
def make_page_url(base: str, page: int) -> str:
    """
    Tạo URL phân trang cho giaoducthoidai.vn
    VD: .../giao-duc/ -> .../giao-duc/?p=2
    """
    if page <= 1: return base
    
    # Xóa tham số p cũ nếu có để tránh trùng (vd: ?p=2&p=3)
    base_clean = re.sub(r'[?&]p=\d+', '', base)
    
    # Kiểm tra xem link gốc đã có tham số nào chưa (?) hay chưa có (&)
    separator = '&' if '?' in base_clean else '?'
    
    return f"{base_clean}{separator}p={page}"

# --- PARSE CHI TIẾT BÀI VIẾT ---
def parse_article(url: str) -> Dict[str, str]:
    result = {
        'title': '', 'summary': '', 'content': '', 'keywords': '',
        'publish_time': '', 'url': url, 'tags': '', 'description': ''
    }
    
    try:
        resp = session.get(url, timeout=15)
        if resp.status_code != 200: return result
        
        resp.encoding = 'utf-8' 
        soup = BeautifulSoup(resp.text, 'html.parser')

        # 1. URL
        meta_url = soup.find('meta', attrs={'name': 'twitter:url'})
        if meta_url: result['url'] = meta_url.get('content', url)

        # 2. Title
        meta_title = soup.find('meta', property='og:title')
        if meta_title: result['title'] = clean_text(meta_title.get('content'))
        
        # 3. Summary
        meta_og_desc = soup.find('meta', property='og:description')
        if meta_og_desc:
            raw_summary = clean_text(meta_og_desc.get('content'))
            result['summary'] = remove_prefix_gdtd(raw_summary)

        # 4. Description
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        if meta_desc: result['description'] = clean_text(meta_desc.get('content'))

        # 5. Keywords
        meta_kw = soup.find('meta', attrs={'name': 'news_keywords'})
        if not meta_kw: meta_kw = soup.find('meta', attrs={'name': 'keywords'})
        if meta_kw: result['keywords'] = clean_text(meta_kw.get('content'))

        # 6. Publish Time
        meta_time = soup.find('meta', property='article:published_time')
        if meta_time: result['publish_time'] = clean_text(meta_time.get('content'))

        # 7. Tags
        meta_tags = soup.find_all('meta', property='article:tag')
        tag_list = [clean_text(t.get('content')) for t in meta_tags if t.get('content')]
        result['tags'] = ', '.join(tag_list)

        # 8. Content
        body = soup.select_one('.article__body.zce-content-body.cms-body')
        
        if body:
            # Xóa tiêu đề con, script, quảng cáo
            for junk in body.find_all(['h1', 'script', 'style', '.relate-news', '.box-ads', '.noted-info', 'figure']):
                junk.decompose()

            paras = body.select('p')
            cleaned_paras = []
            for p in paras:
                txt = clean_text(p.get_text())
                if len(txt) > 2: 
                    cleaned_paras.append(txt)
            
            result['content'] = ' '.join(cleaned_paras)

        return result

    except Exception as e:
        print(f"Lỗi parse: {e}")
        return result

# --- CRAWL UTILS ---
def get_links_extended(url: str) -> List[str]:
    try:
        resp = session.get(url, timeout=15)
        if resp.status_code != 200: return []
        
        soup = BeautifulSoup(resp.text, 'html.parser')
        article_links = set()

        for a in soup.find_all('a', href=True):
            full = normalize_href(a['href'], base=url)
            if not full: continue
            
            # Chỉ lấy link thuộc domain và đúng regex post(\d+).html
            if DOMAIN in full and ARTICLE_URL_RE.search(full):
                # Tránh lấy link trùng với chính trang danh mục (dù regex đã chặn, nhưng cẩn thận)
                if full != url: 
                    article_links.add(full)

        return sorted(list(article_links))
    except Exception:
        return []

# --- MAIN LOGIC ---
def main(topics_dict):
    seen_global = set() # Set chứa tất cả các link đã crawl để tránh trùng lặp
    fieldnames = ['topic', 'title', 'summary', 'description', 'content', 'keywords', 'tags', 'publish_time', 'url']

    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for topic_name, url_data in topics_dict.items():
            print(f"\n ========== CHỦ ĐỀ: {topic_name} ==========")
            list_urls = [url_data] if isinstance(url_data, str) else list(url_data)

            for sub_url in list_urls:
                print(f"   >>> Quét nhánh: {sub_url}")
                page = 1
                consecutive_empty = 0
                
                while page <= MAX_PAGES_PER_LINK and consecutive_empty < STOP_AFTER_EMPTY:
                    # Tạo URL trang 2, 3...
                    p_url = make_page_url(sub_url, page)
                    print(f"      -> Trang {page}: {p_url}")
                    
                    # Lấy danh sách link trên trang
                    article_links = get_links_extended(p_url)
                    
                    # Lọc ra những link chưa từng gặp
                    new_links = [l for l in article_links if l not in seen_global]
                    
                    if not new_links:
                        print("         (Không tìm thấy link mới hoặc trang trùng lặp).")
                        consecutive_empty += 1
                    else:
                        consecutive_empty = 0 # Reset bộ đếm nếu tìm thấy link mới
                        count_ok = 0
                        
                        for link in new_links:
                            seen_global.add(link)
                            data = parse_article(link)
                            
                            if data['title'] and data['content']:
                                writer.writerow({
                                    'topic': topic_name,
                                    'title': data['title'],
                                    'summary': data['summary'],
                                    'description': data['description'],
                                    'content': data['content'],
                                    'keywords': data['keywords'],
                                    'tags': data['tags'],
                                    'publish_time': data['publish_time'],
                                    'url': data['url']
                                })
                                count_ok += 1
                                print(f"         ✅ {data['title'][:50]}...")
                            
                            time.sleep(0.1) # Delay nhỏ để tránh spam server
                        
                        print(f"      -> Đã lưu {count_ok} bài mới.")
                    
                    page += 1
                    time.sleep(SLEEP_BETWEEN_REQUESTS)
                
                if consecutive_empty >= STOP_AFTER_EMPTY:
                    print(f"      [STOP] Dừng vì {STOP_AFTER_EMPTY} trang liên tiếp không có bài mới.")

    print("\n[FINISH] Hoàn tất crawl. File:", OUTPUT_FILE)

# --- DANH MỤC CỦA GIAODUCTHOIDAI.VN ---
# --- DANH MỤC CỦA GIAODUCTHOIDAI.VN ---
if __name__ == "__main__":
    topics = {
        'Giáo dục': ['https://giaoducthoidai.vn/chinh-sach/',
                     'https://giaoducthoidai.vn/dia-phuong/',
                     'https://giaoducthoidai.vn/tuyen-sinh-du-hoc/',
                     'https://giaoducthoidai.vn/giao-duc-bon-phuong/',
                     'https://giaoducthoidai.vn/chuyen-dong/'],
        'Thời sự': ['https://giaoducthoidai.vn/giao-duc-do-thi/',
                    'https://giaoducthoidai.vn/thoi-su-xa-hoi/',
                    'https://giaoducthoidai.vn/chinh-tri/',
                    'https://giaoducthoidai.vn/kinh-te/'],
        "Giáo dục phát luật": ['https://giaoducthoidai.vn/an-ninh/',
                               'https://giaoducthoidai.vn/phap-dinh/',
                               'https://giaoducthoidai.vn/goc-nhin/'],
        "Kết nối": ['https://giaoducthoidai.vn/cong-doan/',
                    'https://giaoducthoidai.vn/dong-hanh/',
                    'https://giaoducthoidai.vn/khoa-hoc/'],
        "Trao đổi": ['https://giaoducthoidai.vn/phuong-phap/',
                     'https://giaoducthoidai.vn/goc-chuyen-gia/'],
        "Học đường": ['https://giaoducthoidai.vn/ky-nang-song/',
                      'https://giaoducthoidai.vn/du-hoc/',
                      'https://giaoducthoidai.vn/guong-mat/',
                      'https://giaoducthoidai.vn/the-chat/'],
        "Nhân ái": 'https://giaoducthoidai.vn/nhan-ai/',
        "Thế giới": ['https://giaoducthoidai.vn/giao-duc-quoc-phong/',
                     'https://giaoducthoidai.vn/the-gioi-do-day/',
                     'https://giaoducthoidai.vn/chuyen-la/'],
        "Sức khỏe": ['https://giaoducthoidai.vn/khoe-dep/',
                     'https://giaoducthoidai.vn/gia-dinh/',
                     'https://giaoducthoidai.vn/day-lui-covid/'],
        "Văn hóa": ['https://giaoducthoidai.vn/sang-tac/',
                    'https://giaoducthoidai.vn/doi-song-van-hoa/',
                    'https://giaoducthoidai.vn/the-gioi-sao/'],
        "Thể thao": 'https://giaoducthoidai.vn/the-thao-hoc-duong/'
    }
    
    main(topics)


 ========== CHỦ ĐỀ: Giáo dục ==========
   >>> Quét nhánh: https://giaoducthoidai.vn/chinh-sach/
      -> Trang 1: https://giaoducthoidai.vn/chinh-sach/
         ✅ 8 nhóm nội dung được tiếp thu hoàn thiện Luật Giáo...
         ✅ Bước tiến rõ rệt từ chính sách dành cho giáo viên,...
         ✅ Chính sách đột phá, thí điểm học vượt cấp, vượt lớ...
         ✅ Chính sách giáo dục có hiệu lực từ tháng 1 năm 202...
         ✅ Công tác pháp chế giáo dục: Khơi thông nguồn lực, ...
         ✅ Đắk Lắk: Khi chính sách giáo dục gắn với trách nhi...
         ✅ Đào tạo nhân lực chất lượng cao người DTTS: Xung l...
         ✅ Đặt tên cơ sở giáo dục đại học và phân hiệu: Chuẩn...
         ✅ Đột phá chính sách đãi ngộ nhà giáo: Đòn bẩy để gi...
         ✅ Đột phá giáo dục và đào tạo TPHCM từ Nghị quyết 71...
         ✅ Dự kiến xét đặc cách thăng tiến nhà giáo: 'Cởi tró...
         ✅ Kỳ vọng vào những đổi thay...
         ✅ Linh hoạt cho trường học, tăng giám sát xã hội quả...
         ✅ Minh bạch 'chư

### Thu thập dữ liệu từ mst.gov.vn

In [19]:
import requests
import re
import csv
import time
import html
from bs4 import BeautifulSoup
from typing import Optional, List, Tuple, Dict
from urllib.parse import urljoin

# --- CẤU HÌNH ---
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/124.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://mst.gov.vn/',
})

DOMAIN = 'https://mst.gov.vn'
# Regex bắt các link bài viết (đuôi .htm)
ARTICLE_URL_RE = re.compile(r'.*\.htm$', re.IGNORECASE)

# --- TUỲ CHỈNH ---
MAX_PAGES_PER_LINK = 40       # Số trang muốn quét mỗi mục
STOP_AFTER_EMPTY = 3        
SLEEP_BETWEEN_REQUESTS = 1.0 
OUTPUT_FILE = 'mst_gov_data.csv'

# --- HÀM TIỆN ÍCH XỬ LÝ TEXT ---
def clean_text(s: str) -> str:
    """Xóa ký tự lạ, decode HTML và đưa về 1 dòng"""
    if not s: return ''
    s = html.unescape(s)
    return re.sub(r'\s+', ' ', s).strip()

def normalize_href(href: str, base: str = DOMAIN) -> Optional[str]:
    if not href: return None
    href = href.strip()
    if href.startswith(('javascript:', '#', 'mailto:')): return None
    if href.startswith('//'): return 'https:' + href
    if href.startswith('/'): return urljoin(DOMAIN, href)
    if href.startswith('http'): return href
    return urljoin(base, href)

# --- THAY ĐỔI CHÍNH Ở ĐÂY ---
def make_page_url(base: str, page: int) -> str:
    """
    Chuyển đổi URL sang dạng phân trang:
    Page 1: .../doi-moi-sang-tao.htm
    Page 2: .../doi-moi-sang-tao/trang-2.htm
    """
    if page <= 1: 
        return base
    
    # 1. Bỏ đuôi .htm hoặc .html ở cuối base url
    base_no_ext = re.sub(r'\.html?$', '', base, flags=re.IGNORECASE)
    
    # 2. Ghép thêm /trang-{page}.htm
    return f"{base_no_ext}/trang-{page}.htm"

# --- PARSE CHI TIẾT BÀI VIẾT ---
def parse_article(url: str) -> Dict[str, str]:
    result = {
        'title': '', 'summary': '', 'content': '', 'keywords': '',
        'publish_time': '', 'url': url, 'tags': '', 'description': ''
    }
    
    try:
        resp = session.get(url, timeout=20)
        if resp.status_code != 200: return result
        
        resp.encoding = 'utf-8' 
        soup = BeautifulSoup(resp.text, 'html.parser')

        # 1. URL (rel="canonical")
        link_canonical = soup.find('link', attrs={'rel': 'canonical'})
        if link_canonical:
            result['url'] = link_canonical.get('href', url)

        # 2. Title (ưu tiên lấy thẻ h1 nếu có)
        div_title = soup.find(class_='btn-group')
        if div_title:
            header_tag = div_title.find(['h1', 'h2', 'h3'])
            if header_tag:
                result['title'] = clean_text(header_tag.get_text())
            else:
                result['title'] = clean_text(div_title.get_text())
        
        if not result['title']:
            meta_title = soup.find('meta', property='og:title')
            if meta_title: result['title'] = clean_text(meta_title.get('content'))

        # 3. Summary
        meta_og_desc = soup.find('meta', property='og:description')
        if meta_og_desc:
            result['summary'] = clean_text(meta_og_desc.get('content'))

        # 4. Description
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        if meta_desc:
            result['description'] = clean_text(meta_desc.get('content'))

        # 5. Keywords
        meta_kw = soup.find('meta', attrs={'name': 'news_keywords'})
        if not meta_kw:
            meta_kw = soup.find('meta', attrs={'name': 'keywords'})
        if meta_kw:
            result['keywords'] = clean_text(meta_kw.get('content'))

        # 6. Public Time
        time_tag = soup.find(attrs={'data-role': 'publishdate'})
        if time_tag:
            result['publish_time'] = clean_text(time_tag.get_text())

        # 7. Content
        body = soup.select_one('.detail-content.afcbc-body')
        
        if body:
            for junk in body.select('script, style, .relate-news, .box-ads'):
                junk.decompose()

            cleaned_paras = []
            paras = body.select('p')
            
            for p in paras:
                if p.find('strong'): continue # Bỏ qua thẻ in đậm
                
                txt = clean_text(p.get_text())
                if len(txt) > 2: 
                    cleaned_paras.append(txt)
            
            result['content'] = ' '.join(cleaned_paras)

        return result

    except Exception as e:
        print(f"Lỗi parse ({url}): {e}")
        return result

# --- CRAWL UTILS ---
def get_links_extended(url: str) -> Tuple[List[str], List[str]]:
    try:
        resp = session.get(url, timeout=20)
        if resp.status_code != 200: return [], []
        
        soup = BeautifulSoup(resp.text, 'html.parser')
        article_links = set()

        for a in soup.find_all('a', href=True):
            full = normalize_href(a['href'], base=url)
            if not full: continue
            
            if DOMAIN in full and ARTICLE_URL_RE.search(full):
                # --- LỌC QUAN TRỌNG ---
                # Loại bỏ link chính nó
                if full == url: continue
                
                # Loại bỏ link phân trang (trang-2.htm, trang-3.htm...) để tránh nhầm là bài viết
                if '/trang-' in full: continue
                
                article_links.add(full)

        return sorted(article_links), [] 
    except Exception:
        return [], []

# --- MAIN LOGIC ---
def main(topics_dict):
    seen_global = set()
    fieldnames = ['topic', 'title', 'summary', 'keywords', 'description', 'content', 'publish_time', 'url']

    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for topic_name, url_data in topics_dict.items():
            print(f"\n ========== CHỦ ĐỀ: {topic_name} ==========")
            list_urls = [url_data] if isinstance(url_data, str) else list(url_data)

            for sub_url in list_urls:
                print(f"   >>> Quét nhánh: {sub_url}")
                page = 1
                consecutive_empty = 0
                
                while page <= MAX_PAGES_PER_LINK and consecutive_empty < STOP_AFTER_EMPTY:
                    # Tạo URL phân trang theo format mới
                    p_url = make_page_url(sub_url, page)
                    print(f"      -> Trang {page}: {p_url}")
                    
                    try:
                        article_links, _ = get_links_extended(p_url)
                    except:
                        article_links = []

                    new_links = [l for l in article_links if l not in seen_global]
                    
                    if not new_links:
                        print("         (Không tìm thấy link mới hoặc hết trang).")
                        consecutive_empty += 1
                    else:
                        consecutive_empty = 0
                        count_ok = 0
                        for link in new_links:
                            seen_global.add(link)
                            data = parse_article(link)
                            
                            if data['title'] and data['content']:
                                writer.writerow({
                                    'topic': topic_name,
                                    'title': data['title'],
                                    'summary': data['summary'],
                                    'keywords': data['keywords'],
                                    'description': data['description'],
                                    'content': data['content'],
                                    'publish_time': data['publish_time'],
                                    'url': data['url']
                                })
                                count_ok += 1
                                print(f"         ✅ {data['title'][:50]}...")
                            time.sleep(0.2) 
                        
                        print(f"      -> Đã lưu {count_ok} bài.")
                    
                    page += 1
                    time.sleep(SLEEP_BETWEEN_REQUESTS)

    print("\n[FINISH] Hoàn tất crawl. File:", OUTPUT_FILE)

# --- DANH MỤC CẦN CRAWL ---
if __name__ == "__main__":
    topics = {
        'Khoa học công nghệ': [
            'https://mst.gov.vn/tin-tuc-su-kien/khoa-hoc-va-cong-nghe.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/doi-moi-sang-tao.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/chuyen-doi-so.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/buu-chinh-vien-thong.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/so-huu-tri-tue.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/tieu-chuan-do-luong-chat-luong.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/nang-luong-nguyen-tu.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/tin-tong-hop.htm',
            'https://mst.gov.vn/tin-tuc-su-kien/tin-dia-phuong.htm'
        ],
    }
    
    main(topics)


 ========== CHỦ ĐỀ: Khoa học công nghệ ==========
   >>> Quét nhánh: https://mst.gov.vn/tin-tuc-su-kien/khoa-hoc-va-cong-nghe.htm
      -> Trang 1: https://mst.gov.vn/tin-tuc-su-kien/khoa-hoc-va-cong-nghe.htm
         ✅ “80 năm – Niềm tin và khát vọng”: Dấu ấn nghệ thuậ...
         ✅ Bộ Khoa học và Công nghệ đẩy mạnh cải cách hành ch...
         ✅ Bộ Khoa học và Công nghệ triển khai đồng bộ các nh...
         ✅ Cán bộ Bộ Khoa học và Công nghệ tăng cường sử dụng...
         ✅ Chương trình “Nghiên cứu khoa học lý luận chính tr...
         ✅ Công bố quyết định điều động, bổ nhiệm Thứ trưởng ...
         ✅ Công bố và trao Giải thưởng "Sản phẩm Công nghệ số...
         ✅ Dấu ấn nổi bật của ngành KH&CN trong năm 2025...
         ✅ Đoàn đại biểu Bộ KH&CN dự Đại hội Thi đua yêu nước...
         ✅ Đưa tổ chức nghiên cứu và phát triển công lập thàn...
         ✅ Khoa học công nghệ, đổi mới sáng tạo và chuyển đổi...
         ✅ Khối Đổi mới sáng tạo: Lấy “toàn dân - hệ sinh thá...
         ✅ Làm 

### Trang baochinhphu.vn

In [5]:
import requests
import re
import csv
import time
import html
from bs4 import BeautifulSoup
from typing import Optional, List, Dict, Tuple
from urllib.parse import urljoin

# --- CẤU HÌNH ---
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/124.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://baochinhphu.vn/',
})

DOMAIN = 'https://baochinhphu.vn'

# Regex: Bắt các link bài viết (thường kết thúc bằng id số và .htm)
ARTICLE_URL_RE = re.compile(r'.*\d+\.htm$', re.IGNORECASE)

# --- TUỲ CHỈNH ---
MAX_PAGES_PER_LINK = 50      # Tăng lên 50 để lấy nhiều tin hơn (tương đương 50 lần ấn xem thêm)
SLEEP_BETWEEN_REQUESTS = 0.5 
OUTPUT_FILE = 'baochinhphu_data.csv' 

# --- HÀM TIỆN ÍCH ---
def clean_text(s: str) -> str:
    if not s: return ''
    s = html.unescape(s)
    return re.sub(r'\s+', ' ', s).strip()

def normalize_href(href: str, base: str = DOMAIN) -> Optional[str]:
    if not href: return None
    href = href.strip()
    if href.startswith(('javascript:', '#', 'mailto:', 'tel:')): return None
    if href.startswith('/'): return urljoin(DOMAIN, href)
    if href.startswith('http'): return href
    return urljoin(base, href)

def make_page_url(base: str, page: int) -> str:
    """Tạo URL phân trang: ?page=2, ?page=3..."""
    if page <= 1: return base
    if '?' in base:
        if re.search(r'[?&]page=\d+', base):
            return re.sub(r'([?&])page=\d+', lambda m: m.group(1) + f'page={page}', base)
        else:
            return f"{base}&page={page}"
    else:
        return f"{base}?page={page}"

# --- PARSE CHI TIẾT BÀI VIẾT ---
def parse_article(url: str) -> Dict[str, str]:
    result = {
        'title': '', 'summary': '', 
        'content': '', 'keywords': '', 'publish_time': '', 'url': url
    }
    
    try:
        resp = session.get(url, timeout=20)
        if resp.status_code != 200: return result
        
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'html.parser')

        # --- META ---
        if soup.title: result['title'] = clean_text(soup.title.string)

        meta_desc = soup.find('meta', property='og:description')
        if meta_desc: result['summary'] = clean_text(meta_desc.get('content'))

        link_canonical = soup.find('link', attrs={'rel': 'canonical'})
        if link_canonical: result['url'] = link_canonical.get('href', url)

        meta_kw = soup.find('meta', attrs={'name': 'news_keywords'})
        if not meta_kw: meta_kw = soup.find('meta', attrs={'name': 'keywords'})
        if meta_kw: result['keywords'] = clean_text(meta_kw.get('content'))

        # --- BODY ---
        time_tag = soup.find(class_='detail-time')
        if time_tag: result['publish_time'] = clean_text(time_tag.get_text())

        body = soup.select_one('.detail-content.afcbc-body.clearfix')
        if body:
            # Xóa rác
            for junk in body.find_all(['img', 'figure', 'figcaption', 'script', 'style', 'table', 'div.box-relate']):
                junk.decompose()
            # Xóa H2 theo yêu cầu
            for h2 in body.find_all('h2'):
                h2.decompose()

            cleaned_paras = []
            paras = body.find_all('p')
            if paras:
                for p in paras:
                    txt = clean_text(p.get_text())
                    if len(txt) > 2: cleaned_paras.append(txt)
                result['content'] = ' '.join(cleaned_paras)
            else:
                result['content'] = clean_text(body.get_text())

        return result
    except Exception as e:
        print(f"Lỗi parse ({url}): {e}")
        return result

# --- CRAWL DANH SÁCH LINK VÀ CHECK NÚT LOADMORE ---
def get_links_and_check_loadmore(url: str) -> Tuple[List[str], bool]:
    """
    Trả về: (Danh sách link bài viết, Có nút 'Xem thêm' hay không?)
    """
    try:
        resp = session.get(url, timeout=20)
        if resp.status_code != 200: return [], False
        
        soup = BeautifulSoup(resp.text, 'html.parser')
        article_links = set()

        # 1. Lấy link bài viết
        for a in soup.find_all('a', href=True):
            full = normalize_href(a['href'], base=url)
            if not full: continue
            
            if DOMAIN in full and ARTICLE_URL_RE.search(full):
                if full.split('?')[0] != url.split('?')[0]: 
                    article_links.add(full)

        # 2. [QUAN TRỌNG] Kiểm tra sự tồn tại của class="loadmore"
        # Nếu tìm thấy class này nghĩa là trang web báo hiệu còn dữ liệu để xem thêm
        has_loadmore = False
        loadmore_btn = soup.select_one('.loadmore')
        
        # Đôi khi nút tồn tại nhưng bị ẩn (display: none), ta kiểm tra kỹ hơn
        if loadmore_btn:
            style = loadmore_btn.get('style', '').lower()
            if 'display: none' not in style:
                has_loadmore = True
        
        return sorted(list(article_links)), has_loadmore

    except Exception:
        return [], False

# --- MAIN LOGIC ---
def main(topics_dict):
    seen_global = set()
    fieldnames = ['topic', 'title', 'summary', 'keywords', 'content', 'publish_time', 'url']

    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for topic_name, url_data in topics_dict.items():
            print(f"\n ========== CHỦ ĐỀ: {topic_name} ==========")
            list_urls = [url_data] if isinstance(url_data, str) else list(url_data)

            for sub_url in list_urls:
                print(f"   >>> Quét nhánh: {sub_url}")
                page = 1
                keep_crawling = True # Biến cờ để kiểm soát vòng lặp dựa trên nút Loadmore
                
                while page <= MAX_PAGES_PER_LINK and keep_crawling:
                    p_url = make_page_url(sub_url, page)
                    print(f"      -> Trang {page}: {p_url}")
                    
                    # Lấy link và trạng thái nút Loadmore
                    new_links_raw, has_loadmore_btn = get_links_and_check_loadmore(p_url)
                    
                    # Cập nhật trạng thái vòng lặp
                    # Nếu không tìm thấy nút loadmore -> Đây là trang cuối -> Dừng sau lần này
                    if not has_loadmore_btn:
                        print("         (Không tìm thấy nút 'Xem thêm', chuẩn bị dừng nhánh này).")
                        keep_crawling = False
                    
                    # Lọc trùng lặp
                    new_links = [l for l in new_links_raw if l not in seen_global]
                    
                    if not new_links:
                        print("         (Không có bài mới).")
                    else:
                        count_ok = 0
                        for link in new_links:
                            seen_global.add(link)
                            data = parse_article(link)
                            
                            if data['title'] and data['content']:
                                writer.writerow({
                                    'topic': topic_name,
                                    'title': data['title'],
                                    'summary': data['summary'],
                                    'keywords': data['keywords'],
                                    'content': data['content'],
                                    'publish_time': data['publish_time'],
                                    'url': data['url']
                                })
                                count_ok += 1
                                print(f"         ✅ {data['title'][:60]}...")
                            time.sleep(0.1) 
                        
                        print(f"      -> Đã lưu {count_ok} bài.")
                    
                    page += 1
                    time.sleep(SLEEP_BETWEEN_REQUESTS)

    print("\n[FINISH] Hoàn tất crawl. File:", OUTPUT_FILE)

# --- DANH MỤC CẦN CRAWL ---
if __name__ == "__main__":
    topics = {
        'Chính trị': ['https://baochinhphu.vn/chinh-tri.htm',
                      'https://baochinhphu.vn/chinh-tri/to-chuc-nhan-su.htm',
                      'https://baochinhphu.vn/chinh-tri/doi-ngoai.htm',
                      'https://baochinhphu.vn/chinh-tri/hoi-nhap.htm',
                      'https://baochinhphu.vn/chi-dao-quyet-dinh-cua-chinh-phu-thu-tuong-chinh-phu.htm'],
        'Kinh tế': ['https://baochinhphu.vn/kinh-te/ngan-hang.htm',
                    'https://baochinhphu.vn/kinh-te.htm',
                    'https://baochinhphu.vn/kinh-te/chung-khoan.htm',
                    'https://baochinhphu.vn/kinh-te/thi-truong.htm',
                    'https://baochinhphu.vn/kinh-te/doanh-nghiep.htm',
                    'https://baochinhphu.vn/kinh-te/khoi-nghiep.htm',
                    'https://baochinhphu.vn/chinh-sach-va-cuoc-song/chinh-sach-moi.htm'],
        'Văn hóa': ['https://baochinhphu.vn/van-hoa.htm',
                    'https://baochinhphu.vn/van-hoa/the-thao.htm',
                    'https://baochinhphu.vn/van-hoa/du-lich.htm'],
        'Xã hội': ['https://baochinhphu.vn/xa-hoi.htm',
                   'https://baochinhphu.vn/xa-hoi/phap-luat.htm',
                   'https://baochinhphu.vn/xa-hoi/y-te.htm',
                   'https://baochinhphu.vn/xa-hoi/doi-song.htm',
                   'https://baochinhphu.vn/xa-hoi/an-sinh-xa-hoi.htm',
                   'https://baochinhphu.vn/xa-hoi/nong-thon-moi.htm'],
        'Khoa giáo': ['https://baochinhphu.vn/khoa-giao.htm',
                      'https://baochinhphu.vn/khoa-giao/giao-duc.htm',
                      'https://baochinhphu.vn/khoa-giao/khoa-hoc-cong-nghe.htm',
                      'https://baochinhphu.vn/khoa-giao/bien-viet-nam.htm'],
        'Quốc tế': ['https://baochinhphu.vn/quoc-te.htm',
                    'https://baochinhphu.vn/quoc-te/viet-nam-asean.htm']
    }
    
    main(topics)


 ========== CHỦ ĐỀ: Chính trị ==========
   >>> Quét nhánh: https://baochinhphu.vn/chinh-tri.htm
      -> Trang 1: https://baochinhphu.vn/chinh-tri.htm
         ✅ Bộ Công an tổ chức lễ xuất quân đảm bảo an ninh, trật tự Đại...
         ✅ Chủ nhật Đỏ vận động được hàng trăm nghìn đơn vị máu cứu ngư...
         ✅ CHUYỂN ĐỔI SỐ: Thấy gì từ trung tâm dữ liệu chính phủ lớn nh...
         ✅ Cổng Thông tin điện tử Chính phủ: “Cầu nối thông tin” với th...
         ✅ Cổng Thông tin điện tử Chính phủ: Lời cảm ơn cho hành trình ...
         ✅ Đẩy mạnh số hóa trong giám sát tàu cá, bảo đảm dữ liệu 'đúng...
         ✅ Điện chia buồn nguyên Chủ tịch Quốc hội Angola qua đời...
         ✅ Đồng chí Phạm Đại Dương được chỉ định giữ chức Bí thư Tỉnh ủ...
         ✅ Đồng chí Phạm Minh Chính chủ trì Hội nghị tổng kết năm 2025,...
         ✅ Giới thiệu Đại tướng Lương Tam Quang ứng cử đại biểu Quốc hộ...
         ✅ Giới thiệu Đại tướng Phan Văn Giang ứng cử đại biểu Quốc hội...
         ✅ Cổng TTĐT Chính p

KeyboardInterrupt: 